# Loan Approval Prediction: 
### Random Forest Modeling

In [1]:
conda install python-graphviz

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
################### Importing Libraries ######################
import pandas as pd

train_df = pd.read_csv('train.csv')
#train_df.info()

In [ ]:
############ Count number of Categorical and Numerical Columns ######################
train_df = train_df.drop(columns=['Loan_ID']) ## Dropping Loan ID
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area','Credit_History','Loan_Amount_Term']
#categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area','Loan_Amount_Term']

print(categorical_columns)
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']
print(numerical_columns)

In [ ]:
#### Analyze values assigned to columns

In [ ]:
### Data Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt


fig,axes = plt.subplots(4,2,figsize=(12,15))
for idx,cat_col in enumerate(categorical_columns):
    row,col = idx//2,idx%2
    sns.countplot(x=cat_col,data=train_df,hue='Loan_Status',ax=axes[row,col])


plt.subplots_adjust(hspace=1)

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(17,5))
for idx,cat_col in enumerate(numerical_columns):
    sns.boxplot(y=cat_col,data=train_df,x='Loan_Status',ax=axes[idx])

print(train_df[numerical_columns].describe())
plt.subplots_adjust(hspace=1)

In [ ]:
#### Encoding categrical Features: ##########
train_df_encoded = pd.get_dummies(train_df,drop_first=True)
train_df_encoded.head()

In [ ]:
########## Split Features and Target Varible ############
dataset_without_loanStatus = train_df_encoded.drop(columns='Loan_Status_Y')
loanStatus = train_df_encoded['Loan_Status_Y']

################# Splitting into Train -Test Data #######
from sklearn.model_selection import train_test_split

dataset_without_loanStatus_train,dataset_without_loanStatus_test,loanStatus_train,loanStatus_test = train_test_split(dataset_without_loanStatus, loanStatus, test_size=0.2,stratify =y,random_state =42)


############### Handling/Imputing Missing values #############
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
imp_train = imp.fit(dataset_without_loanStatus_train)
dataset_without_loanStatus_train = imp_train.transform(dataset_without_loanStatus_train)
dataset_without_loanStatus_test_imp = imp_train.transform(dataset_without_loanStatus_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,f1_score

rf_clf = RandomForestClassifier(n_estimators=100,max_depth=3,min_samples_leaf = 10)
rf_clf.fit(dataset_without_loanStatus_train, loanStatus_train)
loanStatus_pred = rf_clf.predict(dataset_without_loanStatus_train)
print("Train F1 Score ", f1_score(loanStatus_train, loanStatus_pred))
print("Train Accuracy ", accuracy_score(loanStatus_train, loanStatus_pred))

print("Validation Mean F1 Score: ",cross_val_score(rf_clf,dataset_without_loanStatus_train,loanStatus_train,cv=5,scoring='f1_macro').mean())
print("Validation Mean Accuracy: ",cross_val_score(rf_clf,dataset_without_loanStatus_train,loanStatus_train,cv=5,scoring='accuracy').mean())
